In [1]:
# Генерация файлов для Label Studio - функциональный стиль
import json
from typing import List, Dict, Any
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def generate_label_config(output_path: str = 'label_config.xml') -> None:
    """Генерирует XML конфигурацию для Label Studio"""
    config_template = """
    <View>
      <Header value="Выберите категории, которые относятся к комментарию:"/>
      <Text name="text" value="$text"/>
      <Choices name="sentiment" toName="text" choice="multiple" showInLine="true">
        <Choice value="Нравится скорость отработки заявок"/>
        <Choice value="Нравится качество выполнения заявки"/>
        <Choice value="Нравится качество работы сотрудников"/>
        <Choice value="Понравилось выполнение заявки"/>
        <Choice value="Вопрос решен"/>
      </Choices>
    </View>
    """
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(config_template.strip())
    logger.info(f"Конфигурация сохранена в {output_path}")

def prepare_labeling_data(data: 'pd.DataFrame', 
                         text_column: str = 'comment',
                         output_path: str = 'comments_for_labeling.json') -> None:
    """Подготавливает данные для разметки в Label Studio"""
    labeling_data = [
        {
            'data': {'text': row[text_column]},
            'predictions': [{'result': [], 'score': 0}]
        } 
        for _, row in data.iterrows()
    ]
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(labeling_data, f, ensure_ascii=False, indent=2)
    logger.info(f"Данные для разметки сохранены в {output_path}")

# Пример использования
if __name__ == '__main__':
    import pandas as pd
    # Загрузка данных (замените на ваш DataFrame)
    data = pd.DataFrame({'comment': ['Отличная работа!', 'Быстро решили мой вопрос']})
    
    generate_label_config()
    prepare_labeling_data(data)

2025-05-16 18:48:09,709 - INFO - Конфигурация сохранена в label_config.xml
2025-05-16 18:48:09,710 - INFO - Данные для разметки сохранены в comments_for_labeling.json


In [2]:
# Обработка экспорта из Label Studio - функциональный стиль
import json
import pandas as pd
from typing import List, Dict, Any
import logging

logger = logging.getLogger(__name__)

def process_export_file(export_path: str) -> pd.DataFrame:
    """Обрабатывает экспортированные данные из Label Studio"""
    with open(export_path, 'r', encoding='utf-8') as f:
        exported_data = json.load(f)
    
    categories = [
        'Нравится скорость отработки заявок',
        'Нравится качество выполнения заявки',
        'Нравится качество работы сотрудников',
        'Понравилось выполнение заявки',
        'Вопрос решен'
    ]
    
    processed_rows = []
    
    for item in exported_data:
        if not item.get('annotations'):
            continue
            
        text = item['data']['text']
        annotation = item['annotations'][0]
        
        row = {'text': text}
        row.update({cat: 0 for cat in categories})
        
        for result in annotation['result']:
            for choice in result.get('value', {}).get('choices', []):
                if choice in row:
                    row[choice] = 1
        
        processed_rows.append(row)
    
    df = pd.DataFrame(processed_rows)
    logger.info(f"Обработано {len(df)} аннотированных комментариев")
    return df

# Пример использования
if __name__ == '__main__':
    df = process_export_file('exported_labels.json')
    df.to_csv('processed_labels.csv', index=False, encoding='utf-8')
    logger.info("Данные сохранены в processed_labels.csv")

2025-05-16 18:48:09,730 - INFO - Обработано 150 аннотированных комментариев
2025-05-16 18:48:09,738 - INFO - Данные сохранены в processed_labels.csv


In [5]:
# Обучение модели - функциональный стиль с разными моделями (исправленная версия)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import joblib
import logging
from nltk.corpus import stopwords
import nltk

# Загрузка стоп-слов для русского языка
nltk.download('stopwords')
russian_stop_words = stopwords.words('russian')

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_and_prepare_data(filepath: str) -> tuple:
    """Загружает и подготавливает данные"""
    df = pd.read_csv(filepath)
    X = df['text']
    y = df.drop(columns='text')
    return X, y

def train_models(X, y, test_size: float = 0.2) -> dict:
    """Обучает несколько моделей и возвращает результаты"""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    models = {
        'RandomForest': MultiOutputClassifier(RandomForestClassifier(n_estimators=100)),
        'LogisticRegression': OneVsRestClassifier(LogisticRegression(max_iter=1000)),
        'GradientBoosting': MultiOutputClassifier(GradientBoostingClassifier(n_estimators=50)),
        'SVM': OneVsRestClassifier(SVC(kernel='linear', probability=True))
    }
    
    pipelines = {}
    results = {}
    
    for name, model in models.items():
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(
                max_features=5000,
                stop_words=russian_stop_words,  # Используем список стоп-слов
                ngram_range=(1, 2),
                min_df=2
            )),
            ('clf', model)
        ])
        
        logger.info(f"Обучение модели {name}...")
        pipeline.fit(X_train, y_train)
        pipelines[name] = pipeline
        
        y_pred = pipeline.predict(X_test)
        report = classification_report(y_test, y_pred, target_names=y.columns, output_dict=True)
        results[name] = report
        
        logger.info(f"{name} - Точность: {report['micro avg']['precision']:.2f}")
    
    return pipelines, results

def save_best_model(pipelines: dict, results: dict, filename: str = 'best_model.joblib') -> None:
    """Сохраняет лучшую модель по микро-усредненной точности"""
    best_model_name = max(results.keys(), key=lambda k: results[k]['micro avg']['precision'])
    best_model = pipelines[best_model_name]
    
    joblib.dump(best_model, filename)
    logger.info(f"Лучшая модель '{best_model_name}' сохранена в {filename}")

# Пример использования
if __name__ == '__main__':
    X, y = load_and_prepare_data('processed_labels.csv')
    pipelines, results = train_models(X, y)
    save_best_model(pipelines, results)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Silv3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-05-16 18:52:11,204 - INFO - Обучение модели RandomForest...
c:\Users\Silv3\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Silv3\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2025-05-16 18:52:11,601 - INFO - RandomForest - Точность: 0.82
2025-05-16 1